In [ ]:
!pip install pandas meteostat

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from meteostat import Stations, Hourly
from tqdm import tqdm
import warnings
import numpy as np

# 경고 메시지 무시
warnings.filterwarnings('ignore')

print("스크립트를 시작합니다...")

# --- 1. 설정값 ---
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2023, 12, 31, 23, 59)

# 전 세계 관측소 중 몇 개를 무작위로 테스트할지 결정
N_STATION_SAMPLES = 10000

# (변경됨) 목표로 하는 전체 데이터셋 크기
TOTAL_DATASET_SIZE_N = 1000000

# (추가됨) 로컬 시간대 기준으로 필터링할 시간대 (10, 11, 13, 14시)
TARGET_LOCAL_HOURS = [10, 11, 13, 14, 22, 23, 1, 2]

OUTPUT_FILE = "gee_weather_ground_truth_balanced.csv"

# --- 2. 전 세계 관측소 목록 가져오기 ---
print(f"전 세계 {START_DATE.year}년 기준 활성 관측소를 가져옵니다...")
stations = Stations()
stations = stations.fetch()

active_stations = stations[stations['hourly_end'] >= END_DATE]
print(f"활성 관측소 {len(active_stations)}개 확인.")

n_to_sample = min(len(active_stations), N_STATION_SAMPLES)
station_sample = active_stations.sample(n=n_to_sample, random_state=42)
print(f"이 중 {n_to_sample}개 관측소를 샘플링하여 데이터를 수집합니다...")

# --- 3. 경도 기반 시간대 오프셋 계산 함수 ---
def get_utc_offset_from_lon(lon):
    """
    경도를 기반으로 대략적인 UTC 오프셋을 시간 단위로 계산합니다.
    경도 15도당 1시간씩 변화합니다.
    """
    return int(round(lon / 15.0))

# --- 4. 데이터 수집 및 필터링 ---
all_weather_events = []

for index, station in tqdm(station_sample.iterrows(), total=n_to_sample, desc="관측소 데이터 수집 중"):
    station_id = station.name
    lat = station['latitude']
    lon = station['longitude']
    
    # 경도 기반 UTC 오프셋 계산
    utc_offset_hours = get_utc_offset_from_lon(lon)

    try:
        data = Hourly(station_id, START_DATE, END_DATE)
        data = data.fetch()

        if data.empty:
            continue

        # (변경됨) 날씨 코드(coco)가 0보다 큰 경우 (1:맑음 ~ 27:폭풍)
        events = data[data['coco'] > 0] 

        if not events.empty:
            events = events.reset_index()
            
            # UTC 시간을 로컬 시간으로 변환
            events['local_time'] = events['time'].apply(lambda x: x + timedelta(hours=utc_offset_hours))
            events['local_hour'] = events['local_time'].dt.hour
            
            # 로컬 시간대 기준으로 10, 11, 13, 14시만 필터링
            events = events[events['local_hour'].isin(TARGET_LOCAL_HOURS)]
            
            if not events.empty:
                events['id'] = station_id
                events['lat'] = lat
                events['lon'] = lon
                events['time'] = events['time'].dt.tz_localize(None)
                # 모든 기상 데이터 컬럼 포함
                all_weather_events.append(events)

    except Exception as e:
        continue

if not all_weather_events:
    print("오류: 수집된 기상 현상 데이터가 없습니다.")
    exit()

# --- 5. 데이터 통합 및 정제 ---
print("\n데이터 통합 및 정제 중...")
final_df = pd.concat(all_weather_events, ignore_index=True)
final_df.dropna(subset=['coco'], inplace=True)
final_df['coco'] = final_df['coco'].astype(int)

# 컬럼 순서 재배열 (주요 컬럼을 앞으로)
main_cols = ['id', 'time', 'lat', 'lon', 'coco', 'local_hour']
other_cols = [col for col in final_df.columns if col not in main_cols and col != 'local_time']
final_df = final_df[main_cols + other_cols]

print(f"총 {len(final_df)}개의 기상 현상 이벤트를 수집했습니다. (로컬 시간 {TARGET_LOCAL_HOURS}시 필터링 적용)")
print(f"저장되는 컬럼: {list(final_df.columns)}")
print("\n수집된 데이터의 날씨 코드(coco) 분포 (상위 10개):")
print(final_df['coco'].value_counts().head(10))
print("\n로컬 시간대별 데이터 분포:")
print(final_df['local_hour'].value_counts().sort_index())

# --- 6. 날씨 코드별 균형 샘플링 (Stratified Sampling) ---
print(f"\n목표 크기 {TOTAL_DATASET_SIZE_N}에 맞춰 균형 샘플링을 수행합니다...")

# (변경됨) 날씨 코드 종류 수 자동 계산
weather_type_pool_size = final_df['coco'].nunique()

if weather_type_pool_size == 0:
    print("오류: 유니크한 날씨 코드를 찾을 수 없습니다.")
    exit()

# (변경됨) 클래스별 샘플링 개수 자동 계산
# (N / {weather type pool size})
N_SAMPLES_PER_CLASS = int(np.ceil(TOTAL_DATASET_SIZE_N / weather_type_pool_size))

print(f"발견된 날씨 코드 종류: {weather_type_pool_size} 개")
print(f"각 코드별 샘플링 목표 개수: {N_SAMPLES_PER_CLASS} 개 (자동 계산됨)")

def stratified_sample(df, col, n_samples):
    """
    데이터프레임을 'col' 기준으로 그룹화하고, 
    각 그룹에서 n_samples 개수만큼 샘플링합니다. (그룹 크기가 n_samples보다 작으면 모두 반환)
    """
    return df.groupby(col, group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))

balanced_df = stratified_sample(final_df, 'coco', N_SAMPLES_PER_CLASS)
balanced_df = balanced_df.reset_index(drop=True)

print(f"\n균형 샘플링 완료. 최종 {len(balanced_df)}개의 샘플을 확보했습니다.")
print("최종 샘플의 날씨 코드(coco)별 분포:")
print(balanced_df['coco'].value_counts().sort_index())
print("\n최종 샘플의 로컬 시간대별 분포:")
print(balanced_df['local_hour'].value_counts().sort_index())

# --- 7. CSV 파일로 저장 ---
balanced_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n성공! {OUTPUT_FILE} 파일이 저장되었습니다.")
print(f"저장된 컬럼 목록: {list(balanced_df.columns)}")
print("이 CSV 파일을 GEE Asset에 테이블로 업로드하여 사용할 수 있습니다.")


In [ ]:
import pandas as pd

# CSV 파일 읽기
stations_df = pd.read_csv('active_stations_lat_lon.csv')
ground_truth_df = pd.read_csv('gee_weather_ground_truth_balanced.csv')

# lat, lon을 기준으로 ID 매핑 딕셔너리 생성
# 소수점 정밀도를 고려하여 반올림
stations_df['lat_round'] = stations_df['lat'].round(4)
stations_df['lon_round'] = stations_df['lon'].round(4)

# (lat, lon) -> id 매핑 딕셔너리
station_dict = dict(zip(
    zip(stations_df['lat_round'], stations_df['lon_round']), 
    stations_df['id']
))

# ground_truth에도 반올림된 컬럼 생성
ground_truth_df['lat_round'] = ground_truth_df['lat'].round(4)
ground_truth_df['lon_round'] = ground_truth_df['lon'].round(4)

# ID 매핑
ground_truth_df['id'] = ground_truth_df.apply(
    lambda row: station_dict.get((row['lat_round'], row['lon_round']), None),
    axis=1
)

# 임시 컬럼 제거
ground_truth_df = ground_truth_df.drop(['lat_round', 'lon_round'], axis=1)

# 컬럼 순서 재배열 (id를 첫 번째 컬럼으로)
cols = ['id'] + [col for col in ground_truth_df.columns if col != 'id']
ground_truth_df = ground_truth_df[cols]

# 결과 저장
ground_truth_df.to_csv('gee_weather_ground_truth_balanced_with_id.csv', index=False)

# 매칭 결과 확인
print(f"Total rows: {len(ground_truth_df)}")
print(f"Matched rows (with ID): {ground_truth_df['id'].notna().sum()}")
print(f"Unmatched rows (without ID): {ground_truth_df['id'].isna().sum()}")

# 처음 몇 행 출력
print("\n첫 5행:")
print(ground_truth_df.head())